In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.autograd import Variable

import glob
import os
import matplotlib.pyplot as plt
import torchvision.models as models
import cv2
import pickle
import random

In [2]:
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_layers)
        self.fc2 = nn.Linear(hidden_layers, hidden_layers)
        self.fc3 = nn.Linear(hidden_layers, output_size)
    
    def forward(self,data):
        x = F.tanh(self.fc1(data))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x,dim = 1)
        

In [3]:
def label2onehot(label_id):
    vector = np.zeros(101)
    vector[label_id] = 1
    return vector

def onehot2label(vector):
    return np.argmax(vector)

In [4]:
def chunk(data,batch_size):
    batches = []
    x = np.arange(len(data) // batch_size + 2)*batch_size
#     print(x)
    for b,e in zip(x[0:-1],x[1:]):
        batches.append(data[b:e])
    return batches    

In [5]:
def encode_batch(minibatch):
    label_batch = []
    feature_batch_vector = []
    for label, feature in minibatch:
        label_batch.append(label)
        feature_batch_vector.append(feature.squeeze())
    return np.array(label_batch), np.array(feature_batch_vector)
    

In [6]:
def validate(test_data,net):
    correct = 0
    for label, feature in test_data:
        x = Variable(torch.FloatTensor(feature)).cuda()
        output = net(x)
        if onehot2label(output.cpu().data.numpy()) == label:
            correct += 1
    return correct / len(test_data)
        
        

In [7]:
with open("training_data.pkl","rb") as f:
    training_data = pickle.load(f)
    random.shuffle(training_data)
    
with open("test_data.pkl","rb") as f:
    test_data = pickle.load(f)

In [8]:
training_batches = chunk(training_data,128)

In [11]:
net = Classifier(4096,2048,101)
learning_rate = 0.0001
optimizer = optim.Adam(net.parameters(), lr=learning_rate )
loss = nn.CrossEntropyLoss()
net.train()
net.cuda()
max_epoch = 200
for ep in range(max_epoch):
    err_sum = 0 
    for minibatch in training_batches:
        feature_label, feature_vector = encode_batch(minibatch)
        feature_tensor = Variable(torch.FloatTensor(feature_vector)).cuda()
        feature_label = Variable(torch.LongTensor(feature_label)).cuda()
        output = net(feature_tensor)
        error = loss(output, feature_label)
        err_sum += error
        optimizer.zero_grad()
        error.backward()
        optimizer.step()
    avg_err = err_sum.cpu().data.numpy()/ len(minibatch)
    print("ep[%d]\t training error: %f\t test acc: %f"%(ep, avg_err, validate(test_data,net)))

ep[0]	 training error: 26.442166	 test acc: 0.345822
ep[1]	 training error: 25.676837	 test acc: 0.389663
ep[2]	 training error: 25.381498	 test acc: 0.414178
ep[3]	 training error: 25.197162	 test acc: 0.421030
ep[4]	 training error: 25.056172	 test acc: 0.430535
ep[5]	 training error: 24.930965	 test acc: 0.436713
ep[6]	 training error: 24.822737	 test acc: 0.437465
ep[7]	 training error: 24.735093	 test acc: 0.438455
ep[8]	 training error: 24.658201	 test acc: 0.437545
ep[9]	 training error: 24.596630	 test acc: 0.439604
ep[10]	 training error: 24.544236	 test acc: 0.446257
ep[11]	 training error: 24.496232	 test acc: 0.447485
ep[12]	 training error: 24.447315	 test acc: 0.450337
ep[13]	 training error: 24.402193	 test acc: 0.454535
ep[14]	 training error: 24.353456	 test acc: 0.453030
ep[15]	 training error: 24.303230	 test acc: 0.452317
ep[16]	 training error: 24.254732	 test acc: 0.452079
ep[17]	 training error: 24.201998	 test acc: 0.461861
ep[18]	 training error: 24.160388	 tes

ep[151]	 training error: 23.009546	 test acc: 0.492990
ep[152]	 training error: 23.006380	 test acc: 0.492634
ep[153]	 training error: 23.006577	 test acc: 0.490495
ep[154]	 training error: 23.000062	 test acc: 0.491525
ep[155]	 training error: 22.990424	 test acc: 0.493861
ep[156]	 training error: 22.983320	 test acc: 0.492871
ep[157]	 training error: 22.977463	 test acc: 0.487683
ep[158]	 training error: 22.977561	 test acc: 0.492634
ep[159]	 training error: 22.968180	 test acc: 0.490059
ep[160]	 training error: 22.966180	 test acc: 0.489743
ep[161]	 training error: 22.960380	 test acc: 0.496238
ep[162]	 training error: 22.957596	 test acc: 0.490139
ep[163]	 training error: 22.962553	 test acc: 0.488673
ep[164]	 training error: 22.954960	 test acc: 0.493703
ep[165]	 training error: 22.950740	 test acc: 0.496238
ep[166]	 training error: 22.946787	 test acc: 0.495802
ep[167]	 training error: 22.947275	 test acc: 0.487881
ep[168]	 training error: 22.951317	 test acc: 0.490059
ep[169]	 t